In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv

In [ ]:
import pickle
from collections import defaultdict
from tqdm import tqdm
from glob import glob
DATA_DIR = "./data_2/"
data_files = [file for file in glob(DATA_DIR+"results_*.pkl")]
data = defaultdict(int)
for dataset in tqdm(data_files):
    start = dataset.find('A')
    end = dataset.find('.pkl')
    id = dataset[start:end]
    with open(dataset,"rb") as f:
       dataset = pickle.load(f)
       data[id] = dataset

with open('alldata.pkl',"wb") as f:
    pickle.dump(data,f)
    


In [35]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.1/repo.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.dgl.ai/wheels/torch-2.1/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 14.8 MB/s eta 0:00:00a 0:00:01


In [2]:
import pickle
with open("alldata.pkl","rb") as f:
       data = pickle.load(f)

/share/pkg.8/academic-ml/fall-2024/install/fall-2024-pyt/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(i

In [59]:
data = []
for (id,sample) in processed_samples.items():
    data.append(list(sample.values()))


list

In [3]:
import numpy as np
import scipy.sparse as sp
import dgl
import torch
import torch.nn as nn
import pickle
from collections import defaultdict
from glob import glob
from tqdm import tqdm

def np_to_dgl(np_adj):
    # Convert to SciPy sparse matrix
    sparse_matrix = sp.coo_matrix(np_adj)

    # Convert to DGL graph
    g = dgl.from_scipy(sparse_matrix)

    return g

def add_embedding_features(g,features,sample_id):
    try:
        (num_nodes,num_dims) = features.shape

        initial_features = torch.zeros((num_nodes, num_dims))  # All features initialized to 0
        g.ndata['feat'] = initial_features

        g.ndata['feat'] = features

        return g
    except Exception as e:
        
        print(f'error adding embeddings as node features {e} with {sample_id}')

def layer_to_graph(embeddings,adj,sample_id):
    g = np_to_dgl(adj)

    return add_embedding_features(g,embeddings,sample_id)


def embed_label(labels,embedding_dim):
    """
    labels: str list
    
    """
    # Step 1: Map string labels to integer indices
    label_to_index = {label: idx for idx, label in enumerate(set(labels))}
    encoded_labels = [label_to_index[label] for label in labels]

    # Step 2: Create a learnable embedding layer
    embedding_dim = 4  # Desired embedding dimension
    num_labels = len(label_to_index)  # Number of unique labels
    
    # Create an embedding layer
    embedding_layer = nn.Embedding(num_labels, embedding_dim)
    
    # Step 3: Use the embedding layer to get compressed vectors
    indices = torch.tensor(encoded_labels)
    embedded_vectors = embedding_layer(indices)

    return embedded_vectors

def parse_labels(labels):
    new_labels = [[] for _ in range(4)]
    for (id,pos,relations) in labels:
        if "<" in id or ">" in id:
            id = ""
        if type(relations) != tuple:
            relations = ("","")
        relation_type = relations[0]
        relation = relations[1]
        new_labels[0].append(id)
        new_labels[1].append(pos)
        new_labels[2].append(relation_type)
        new_labels[3].append(relation)
    return new_labels

with open("alldata.pkl","rb") as f:
       data = pickle.load(f)

processed_samples = defaultdict(lambda: defaultdict(int))

for (paper,samples) in tqdm(data.items()):
    for (i,sample) in enumerate(samples):
        [text,labels,layers] = sample
        
        sample_id = paper+"-"+str(i)
        
        [ids,poss,relation_type,relation] = parse_labels(labels)

        processed_samples[sample_id]['label'] = list(zip(
            embed_label(ids,13),
            embed_label(poss,6),
            embed_label(relation_type,3),
            embed_label(relation,13)))
        i = 0
        for (layer,v) in layers.items():
            if layer != 'masks':
                embeds = v[0]
                adj = v[1]
                """
                if len(embeds) != len(adj) and i<5:
                    print(sample_id)
                    print(text)
                    i+=1
                    """
                processed_samples[sample_id][layer] = layer_to_graph(embeds,adj,sample_id)
       

with open("processed_samples.pkl","wb") as f:
    pickle.dump(processed_samples,f)





  5%|▌         | 2/40 [00:00<00:06,  5.90it/s]

error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 285 and 292 instead. with A39-23
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 300 and 306 instead. with A39-24
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 295 and 296 instead. with A39-27
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 286 and 295 instead. with A39-29
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 204 and 341 instead. with A39-43
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 347 and 358 instead. with A39-44
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 336 and 371 instead. with A39-45
error adding embeddings as 

 12%|█▎        | 5/40 [00:00<00:05,  6.30it/s]

error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 236 and 248 instead. with A27-38
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 283 and 285 instead. with A24-4
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 275 and 279 instead. with A24-5
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 297 and 302 instead. with A24-6
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 291 and 297 instead. with A24-7
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 301 and 303 instead. with A24-11


 18%|█▊        | 7/40 [00:01<00:04,  7.60it/s]

...

 90%|█████████ | 36/40 [00:05<00:00,  9.65it/s]

error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 283 and 284 instead. with A40-0
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 363 and 370 instead. with A40-17
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 280 and 289 instead. with A16-11
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 364 and 366 instead. with A16-28
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 370 and 372 instead. with A16-29
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 342 and 355 instead. with A34-17


100%|██████████| 40/40 [00:06<00:00,  6.50it/s]


error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 328 and 329 instead. with A12-28
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 314 and 316 instead. with A12-30
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 251 and 283 instead. with A05-20
error adding embeddings as node features Expect number of features to match number of nodes (len(u)). Got 288 and 290 instead. with A05-21


PicklingError: Can't pickle <function <lambda> at 0x148292d79b20>: attribute lookup <lambda> on __main__ failed

In [24]:
#emb,adj = data["A05"][20][2]['layer_1']

In [28]:
data["A05"][20][0]

'n . \uf8ee \uf8ef \uf8f0 (T (T 2,e 2,e 1 k − − . . . T T 1,e 1,e k 1 )v )v d d ··· ··· .. . (T (T n,e n,e 1 k − − . . . T T 1,e 1,e k 1 )v )v d d \uf8f9 \uf8fb \uf8fa \uf8f0 \uf8ee \uf8ef \uf8ef w w w . . . 2 n 3 \uf8fa \uf8fb \uf8fa \uf8f9 = \uf8ef \uf8f0 \uf8ee v v e e 1 k − − T T . . . 1,e 1,e 1 k v v d d \uf8f9 \uf8fa \uf8fb 5 To appear at SIGGRAPH 2003 The matrix used to solve for vertex positions is as follows. \uf8ee \uf8f0 \uf8ef ∑ ∑ i=1 n i=1 n w w . . . i i T T i,e i,e 1 k \uf8f9 \uf8fa \uf8fb v d = \uf8f0 \uf8ef \uf8ee v v . . . e e 1 k \uf8fa \uf8fb \uf8f9 To handle homogeneous coordinates, the translation parts of the ∑ i=1 n w i T i,e k matrices are subtracted from the v e on the right hand side. We solve these least-squares problems using the singular value decomposition. This lets us detect when our matrices are rank deficient, leading to overfitting. We detect this by comparing the ratio of the largest singular value to the smallest, and issuing a warning if there are

In [27]:
adj.shape

(283, 283)

In [ ]:

import spacy

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h



Example HGT model

In [ ]:
import torch
import torch.nn as nn
from dgl.nn import HGTLayer

class TwoLayerHGT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads, num_node_types, num_edge_types, dropout=0.2):
        """
        Parameters:
        - in_dim: Input feature dimension for each node type.
        - hidden_dim: Dimension of hidden representations.
        - out_dim: Dimension of output features.
        - num_heads: Number of attention heads in HGT layers.
        - num_node_types: Number of unique node types.
        - num_edge_types: Number of unique edge types.
        - dropout: Dropout rate.
        """
        super(TwoLayerHGT, self).__init__()
        self.node_type_emb = nn.Embedding(num_node_types, hidden_dim)  # Node type embeddings
        self.input_proj = nn.Linear(in_dim, hidden_dim)  # Project input to hidden_dim
        
        # Two HGT layers
        self.hgt_layer1 = HGTLayer(hidden_dim, hidden_dim, num_heads, num_node_types, num_edge_types, dropout)
        self.hgt_layer2 = HGTLayer(hidden_dim, hidden_dim, num_heads, num_node_types, num_edge_types, dropout)
        
        self.output_proj = nn.Linear(hidden_dim, out_dim)  # Project hidden to output_dim
        self.dropout = nn.Dropout(dropout)

    def forward(self, g, node_features):
        """
        Parameters:
        - g: Heterogeneous DGLGraph.
        - node_features: Dictionary of input node features for each node type.
          Example: {'type1': tensor, 'type2': tensor, ...}
        """
        # Project input features to a common hidden space
        h = {
            ntype: self.input_proj(node_features[ntype])
            for ntype in g.ntypes
        }
        
        # Add node type embeddings
        for ntype in g.ntypes:
            h[ntype] += self.node_type_emb.weight[g.get_ntype_id(ntype)]
        
        # First HGT layer
        h = self.hgt_layer1(g, h)
        h = {k: torch.relu(v) for k, v in h.items()}  # Apply non-linearity
        
        # Second HGT layer
        h = self.hgt_layer2(g, h)
        h = {k: torch.relu(v) for k, v in h.items()}  # Apply non-linearity
        
        # Output projection
        h = {
            ntype: self.output_proj(self.dropout(h[ntype]))
            for ntype in g.ntypes
        }
        return h

# Example usage:
# Initialize the model
model = TwoLayerHGT(
    in_dim=128, 
    hidden_dim=64, 
    out_dim=32, 
    num_heads=4, 
    num_node_types=3, 
    num_edge_types=4, 
    dropout=0.2
)

# Example graph and input features
import dgl
import torch

# Define a simple heterogeneous graph
graph_data = {
    ('paper', 'cites', 'paper'): (torch.tensor([0, 1]), torch.tensor([1, 2])),
    ('author', 'writes', 'paper'): (torch.tensor([0, 1]), torch.tensor([1, 2])),
}
g = dgl.heterograph(graph_data)

# Example node features
node_features = {
    'paper': torch.randn(3, 128),  # 3 papers with 128-dimensional features
    'author': torch.randn(2, 128),  # 2 authors with 128-dimensional features
}

# Forward pass
output = model(g, node_features)
dgl.save_graphs("graph.dgl", g)
print(output)

Training function

In [ ]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0
    """
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    """
    for e in range(20):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In [5]:
with open("lst_samples.pkl","rb") as f:
    data = pickle.load(f)

EOFError: Ran out of input

In [11]:
import dgl

def add_self_loops_to_graphs(graphs):
    """
    Add self-loops to a list of DGL graphs.
    Parameters:
    - graphs: List of DGLGraph objects.
    Returns:
    - List of DGLGraph objects with self-loops added.
    """
    return [dgl.add_self_loop(g) for g in graphs]
total_samples = 0
problems = 0
filtered_data = []
for (y,datapoint) in enumerate(data):
    if len(datapoint) == 4:
        for i in range(1, 4):  # g1, g2, g3
            if isinstance(datapoint[i], dgl.DGLGraph):  # Ensure the graph exists
                datapoint[i] = dgl.add_self_loop(datapoint[i])
            else:
                #print(f"{i}{y}")
                problems+=1
        filtered_data.append(datapoint)
    total_samples+=1



In [208]:
[

1026

In [12]:
def check_graphs(gs):
    for g in gs:
        if not(isinstance(g, dgl.DGLGraph)):
            return False
    return True

x = [check_graphs([g1,g2,g3]) for [label,g1,g2,g3] in filtered_data]

In [13]:
second_filter = []
for i,b in enumerate(x):
    if b:
        second_filter.append(filtered_data[i])

In [14]:
def check_labels(label):
    if len(label) == 250:
        return True
    return False

x = [check_labels(label) for [label,g1,g2,g3] in second_filter]

In [15]:
third_filter = []
for i,b in enumerate(x):
    if b:
        third_filter.append(second_filter[i])

In [54]:
def check_graph_features(graph_list):
    for g in graph_list:
        if not('feat' in g.ndata):
            return False
    return True

checks = [i for (i,[_,g1,g2,g3]) in enumerate(third_filter) if check_graph_features([g1,g2,g3])]

991

In [73]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
import dgl
from dgl.nn import GraphConv
from dgl.dataloading import GraphDataLoader
from torch.nn.functional import binary_cross_entropy_with_logits as loss_fn

class GCNEncoder_vanilla(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNEncoder, self).__init__()
        self.gcn1 = GraphConv(input_dim, hidden_dim)
        self.gcn2 = GraphConv(hidden_dim, output_dim)

    def forward(self, graph):
        if not isinstance(graph, dgl.DGLGraph):
            raise TypeError("Input must be a DGLGraph.")
        if 'feat' not in graph.ndata:
            raise KeyError("Graph must have node features in graph.ndata['feat']")

        x = graph.ndata['feat']  # Node features
        x = F.relu(self.gcn1(graph, x))
        x = self.gcn2(graph, x)
        graph.ndata['h'] = x
        return dgl.mean_nodes(graph, 'h')  # Mean pooling for graph-level output


import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import dropout_adj

from dgl.nn import GraphConv, GATConv

class BATCHGAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super(BATCHGAT, self).__init__()
        
        # Graph Convolutional Layers with Batch Normalization
        self.gcn1 = GraphConv(in_channels, hidden_channels)
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        
        self.gcn2 = GraphConv(hidden_channels, hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)
        
        # Graph Attention Layer
        self.gat1 = GATConv(hidden_channels, hidden_channels, num_heads=8)
        
        # Final Layer for Output
        self.fc = nn.Linear(hidden_channels * 8, out_channels)  # *8 due to multi-head attention
        
        # Dropout Rate
        self.dropout = dropout
    
    def forward(self, graph, x):
        # Apply Graph Convolution with Batch Normalization + Activation + Dropout
        x = F.relu(self.bn1(self.gcn1(graph, x)))
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Another Graph Convolution Layer
        x = F.relu(self.bn2(self.gcn2(graph, x)))
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Apply Graph Attention Layer
        x = self.gat1(graph, x)
        
        # Apply Dropout on attention outputs (here's where we handle dropout in the attention heads)
        x = F.dropout(x, p=self.dropout, training=self.training)
        #x = x.mean(dim=2)
        # Final Classification Layer
        x = self.fc(x)
        
        return F.log_softmax(x, dim=1) # Log-Softmax for multi-class classification


class Decoder(torch.nn.Module):
    def __init__(self, input_dim, output_dim, list_length=250):
        super(Decoder, self).__init__()
        self.list_length = list_length
        self.linear = Linear(input_dim, output_dim * list_length)  # Adjust to correct dimensions

    def forward(self, graph_embeddings):
        """
        Decode concatenated graph embeddings into structured outputs.
        """
        batch_size = graph_embeddings.shape[0]
        linear_output = self.linear(graph_embeddings)  # Shape: (batch_size, list_length * output_dim)
        linear_output = linear_output.view(batch_size, self.list_length, -1)  # Reshape to (batch_size, list_length, output_dim)
        return linear_output



def process_datapoint(datapoint, model):
    """
    Process a single datapoint from the dataset.

    Parameters:
    - datapoint: A list where:
        - datapoint[0] is a 250-length list of 4-element tuples of 4D tensors.
        - datapoint[1], datapoint[2], and datapoint[3] are DGL graphs.

    Returns:
    - target_tensor: Tensor of shape (250, 4).
    - batched_graphs: Combined embedding of g1, g2, g3.
    """
    try:
        # Unpack the datapoint
        target, g1, g2, g3 = datapoint

        # Ensure node features exist for all graphs
        for graph in [g1, g2, g3]:
            #if 'feat' not in graph.ndata:
                #graph.ndata['feat'] = torch.randn(graph.num_nodes(), model.gcn1.in_feats)  # Random features fallback
                # Detach node features to ensure no gradient dependencies
            graph.ndata['feat'] = graph.ndata['feat'].detach()

        # Convert the target (list of 250 tuples) into a tensor of shape (250, 4)
        # Each tuple contains four tensors of length 4, so we concatenate them into one tensor
        target_tensor = torch.stack(
            [torch.cat(tup, dim=0) for tup in target]
        ).detach()  # Shape: (250, 4)

        # Batch the three graphs into one
        batched_graphs = dgl.batch([g1, g2, g3])

        return target_tensor, batched_graphs

    except Exception as e:
        print(f"Error processing datapoint: {e}")
        raise




def train(model, decoder, optimizer, train_data, val_data, list_length, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        decoder.train()
        log=0
        train_loss = 0  # Reset train loss for each epoch

        # Training loop
        for datapoint in tqdm(train_data):
            optimizer.zero_grad()  # Reset gradients for each batch

            target, batched_graphs = process_datapoint(datapoint, model)

            # Forward pass
            graph_embeddings = model(batched_graphs,batched_graphs.ndata['feat'])  # Output: (3, 64)
            graph_embeddings = graph_embeddings.view(1, -1)  # Reshape to (batch_size, input_dim * 3), e.g., (1, 192)

            train_outputs = decoder(graph_embeddings)  # Shape: (batch_size, list_length, 4)

            # Prepare predictions and targets
            predictions = train_outputs[0]  # Use the first batch (since batch_size=1)
            assert predictions.shape == target.shape, f"Shape mismatch: {predictions.shape} vs {target.shape}"

            # Calculate and backpropagate loss
            batch_loss = loss_fn(predictions, target)
            batch_loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            train_loss += batch_loss.item()  # Accumulate scalar loss

        # Validation
        val_loss = 0
        model.eval()
        decoder.eval()
        with torch.no_grad():
            for datapoint in val_data:
                target, batched_graphs = process_datapoint(datapoint, model)
                graph_embeddings = model(batched_graphs).view(1, -1)  # Reshape for decoder
                val_outputs = decoder(graph_embeddings)

                predictions = val_outputs[0]  # Shape: (list_length, 4)
                assert predictions.shape == target.shape, f"Shape mismatch: {predictions.shape} vs {target.shape}"
                val_loss += loss_fn(predictions, target).item()

        print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

from sklearn.model_selection import train_test_split

def split_data(data, val_ratio=0.2, random_state=42):
    """
    Split a dataset into training and validation sets.

    Parameters:
    - data (list): The dataset to be split (list of datapoints).
    - val_ratio (float): Fraction of the data to use as validation (default is 0.2).
    - random_state (int): Random seed for reproducibility (default is 42).

    Returns:
    - train_data (list): Training data split.
    - val_data (list): Validation data split.
    """
    train_data, val_data = train_test_split(data, test_size=val_ratio, random_state=random_state)
    return train_data, val_data

train_data,val_data = split_data(third_filter)
learning_rate = 0.001
list_length=250
num_epochs = 20
input_dim = 768
hidden_dim=64
output_dim=192
dropout=0.5
#model = GCNEncoder(input_dim=768, hidden_dim=192, output_dim=64)
model = BATCHGAT(input_dim, hidden_dim, output_dim,dropout=dropout)
decoder = Decoder(input_dim=192, output_dim=16)
optimizer = torch.optim.Adam(list(model.parameters()) + list(decoder.parameters()), lr=learning_rate)

train(model, decoder, optimizer, train_data, val_data, list_length, num_epochs)


  0%|          | 0/792 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2712x64 and 512x192)

Basic Logical Rules
All own claim nodes must have a support connection from a data node, this is not true of background claims
Acyclic - a node cannot both attack and support the same node at the same time

In [10]:
data = []
for (id,sample) in processed_samples.items():
    data.append(list(sample.values()))



4

In [151]:
def is_graph(x):
    return isinstance(x, dgl.DGLGraph)

def filter_sample(sample):
    [label,g1,g2,g3] = sample
    for g in [g1,g2,g3]:
        if is_graph(g):
            return False
    return True


list(filter(filter_sample,filtered_data))

[]